In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
DATA_PATH = os.path.join('./Datasets') 

# Actions that we try to detect
actions = np.array(['Normal'])

# Thirty videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 60

# Folder start


In [3]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout,GRU
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
DATA_PATH="./Datasets/"
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [6]:
X = np.array(sequences)
print(X.shape)
y = to_categorical(labels).astype(int)

(10, 60, 4)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
# model.add(LSTM(units=8, input_shape=(60, 4)))
model.add(LSTM(units=8, input_shape=(60, 4), kernel_regularizer='l2', recurrent_regularizer='l2', bias_regularizer='l2', return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(actions.shape[0], activation='sigmoid'))
model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=40,batch_size=32, callbacks=[tb_callback],validation_data=(X_test,y_test))
# model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/40
1/1 [==============================] - 3s 3s/step - loss: 0.9076 - accuracy: 0.5729 - val_loss: 0.8752 - val_accuracy: 0.7917
Epoch 2/40
1/1 [==============================] - 0s 64ms/step - loss: 0.9097 - accuracy: 0.5917 - val_loss: 0.8684 - val_accuracy: 0.8083
Epoch 3/40
1/1 [==============================] - 0s 77ms/step - loss: 0.9109 - accuracy: 0.5542 - val_loss: 0.8615 - val_accuracy: 0.8333
Epoch 4/40
1/1 [==============================] - 0s 74ms/step - loss: 0.8956 - accuracy: 0.6208 - val_loss: 0.8547 - val_accuracy: 0.8500
Epoch 5/40
1/1 [==============================] - 0s 63ms/step - loss: 0.8848 - accuracy: 0.6396 - val_loss: 0.8479 - val_accuracy: 0.8750
Epoch 6/40
1/1 [==============================] - 0s 53ms/step - loss: 0.8761 - accuracy: 0.6542 - val_loss: 0.8411 - val_accuracy: 0.9250
Epoch 7/40
1/1 [==============================] - 0s 56ms/step - loss: 0.8756 - accuracy: 0.6833 - val_loss: 0.8342 - val_accuracy: 0.9250
Epoch 8/40
1/1 [=============